# Financial Crime Detection Demo
## Open-Source Risk Framework

![Risk Framework Logo](https://via.placeholder.com/150x50?text=Risk+Framework)

This notebook demonstrates:
1. Transaction data loading and preprocessing
2. Anomaly detection using Isolation Forest
3. Risk reporting and MCP documentation

## 1. Setup Environment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml

# Framework imports
from risk_framework.models.fincrime_aml_kyc.isolation_forest import IsolationForestRiskDetector
from risk_framework.reporting.report_generator import RiskReportGenerator
from risk_framework.mcp.validate_mcp import MCPValidator

%matplotlib inline
plt.style.use('ggplot')

# Configuration
DATA_PATH = Path("../data/sample_transactions.csv")
REPORT_DIR = Path("../reports")
MODEL_DIR = Path("../models")

## 2. Load and Inspect Data

In [ ]:
transactions = pd.read_csv(DATA_PATH, parse_dates=['transaction_time'])

print(f"Data shape: {transactions.shape}")
print("\nSample data:")
display(transactions.head(3))

print("\nSummary statistics:")
display(transactions.describe())

## 3. Feature Engineering

In [ ]:
# Select and transform features
features = transactions[[
    'amount',
    'duration',
    'customer_age',
    'transaction_hour'
]].copy()

# Log transform monetary values
features['amount_log'] = np.log1p(features['amount'])

# Normalize temporal features
features['hour_sin'] = np.sin(2 * np.pi * features['transaction_hour']/24)
features['hour_cos'] = np.cos(2 * np.pi * features['transaction_hour']/24)

final_features = features.drop(['amount', 'transaction_hour'], axis=1)

print("Final feature set:")
display(final_features.head())

## 4. Anomaly Detection

In [ ]:
# Initialize and train model
model = IsolationForestRiskDetector(
    features=final_features.columns.tolist(),
    contamination=0.01,
    risk_threshold=-0.5,
    model_dir=MODEL_DIR
)

model.fit(final_features, save_model=True)

# Generate predictions
results = model.predict(transactions)
risky_txns = results[results['is_risk'] == 1]

print(f"\nDetected {len(risky_txns):,} risky transactions ({len(risky_txns)/len(results):.2%})")
display(risky_txns.head())

## 5. Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Score distribution
ax1.hist(results['risk_score'], bins=50, alpha=0.7)
ax1.axvline(x=500, color='r', linestyle='--', label='Risk Threshold')
ax1.set_title('Risk Score Distribution')
ax1.set_xlabel('Score')
ax1.set_ylabel('Count')
ax1.legend()

# Feature importance
model.plot_feature_importance(top_n=5, ax=ax2)
ax2.set_title('Top Risk Drivers')

plt.tight_layout()
plt.show()

## 6. Generate Reports

In [ ]:
# Prepare report data
report_data = risky_txns[[
    'transaction_id',
    'amount',
    'customer_id',
    'risk_score',
    'risk_severity'
]].sort_values('risk_score', ascending=False)

metrics = {
    'Total Transactions': len(results),
    'Flagged Transactions': len(risky_txns),
    'Detection Rate': f"{len(risky_txns)/len(results):.2%}",
    'Mean Amount (Flagged)': f"${risky_txns['amount'].mean():,.2f}",
    'Max Risk Score': risky_txns['risk_score'].max()
}

# Generate HTML report
REPORT_DIR.mkdir(exist_ok=True)
reporter = RiskReportGenerator(output_dir=REPORT_DIR)
report_path = reporter.generate_html_report(
    title="Financial Crime Alert Report",
    summary=metrics,
    data=report_data,
    filename="fraud_detection_report"
)

print(f"Report generated: {report_path}")

## 7. Model Documentation (MCP)

In [ ]:
mcp_content = f"""
# Model Control Policy
model:
  name: "isolation_forest_fraud_detector"
  version: "1.0.0"
  type: "unsupervised_anomaly_detection"
  owner: "Financial Crime Team"
  
training:
  features: {final_features.columns.tolist()}
  contamination: 0.01
  risk_threshold: -0.5
  training_date: "{pd.Timestamp.now().date().isoformat()}"
  
monitoring:
  metrics:
    - "feature_drift"
    - "risk_score_distribution"
    - "alert_volume"
  frequency: "daily"
  
validation:
  last_validated: ""
  validation_metrics:
    precision: "TBD"
    recall: "TBD"

approvals:
  - role: "Model Owner"
    name: ""
    date: ""
"""

# Save and validate
mcp_path = Path("../mcp/fraud_detector_mcp.yaml")
mcp_path.parent.mkdir(exist_ok=True)

with open(mcp_path, 'w') as f:
    f.write(mcp_content)

print(f"MCP saved to: {mcp_path}\n")
print(mcp_content)

## Next Steps

1. Review the generated report: `reports/fraud_detection_report.html`
2. Examine model outputs in `results` DataFrame
3. Complete the MCP validation section
4. Schedule monitoring jobs